In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 KB 47.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [ ]:
!pip install db-sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for db-sqlite3: filename=db_sqlite3-0.0.1-py3-none-any.whl size=1794 sha256=fc48353ace0f07bc4d745e9fb771f32270d38f26bf6c22ae4fdf419cbabd1c12
  Stored in directory: /root/.cache/pip/wheels/20/fb/1b/2ba46d337e706f0315053f19c8d6e83fc2431c41b7525caaa2
  Created wheel for db: filename=db-0.1.1-py3-none-any.whl size=3891 sha256=1be21e2ee9a8bb6cb3db93dbea754abad5ca592e7d9740e109cf6e0203470a65
  Stored in directory: /root/.cache/pip/wheels/85/9b/62/ebd5df18a3e9d5ead2b27c11d493f87aedc9d5e3bb0539010c
  Created wheel for antiorm: filename=antiorm-1.2.1-py3-none-any.whl size=31677 sha256=65c5e6c47b80758dd37745729199f2f80a727a848daeaea571fcb20981791df5
  Stored in directory: /root/.cac

In [ ]:
%%writefile app.py
import streamlit as st
import easyocr
import re
import tempfile
import sqlite3
import pandas as pd
from PIL import Image

st.header('Project of Business Card Extraction')

uploaded_file = st.file_uploader("Choose an image...",  type=["png", "jpeg", "jpg"])
# If an image was uploaded, display it and its name
if uploaded_file is not None:
    image = Image.open(uploaded_file)  
    # Display the image in Streamlit
    st.image(image, caption="Uploaded image", use_column_width=True)
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        file_path = tmp_file.name
    reader = easyocr.Reader(['en'])
    result = reader.readtext(file_path, detail=0)
    list = result
    num = '(\+||s)(\d+\-)'
    mail = '\w+@\w+\.com'
    website = '\w+(.com|com)'
    add = '(\d+\w)'
    pin = '(\w+)\s(\d{6})'
    f = []
    for i in list:
       if re.match(num,i) or re.findall(mail,i) or re.findall(website,i) or re.match(add,i) or re.match(pin,i) :
          f.append(i)
    number = []
    web = []
    mailid = []
    address = []
    state = []
    for i in list:
      if re.findall(num,i):
         number.append(i)
      elif re.findall(mail,i):
         mailid.append(i)
      elif re.findall(website,i):
         web.append(i)
      elif re.match(add,i):
         address.append(i)
      elif re.match(pin,i):
         state.append(i)
    if state == []:
       address = ''.join(address)
       parts = address.split(';')
       address = parts[0]
       state = parts[1]
    number = ''.join(number)
    web = ''.join(web)
    mailid = ''.join(mailid)
    address = ''.join(address)
    state = ''.join(state)
    f2 =[]
    for i in list:
      if i not in f:
        f2.append(i)
    name = f2[0]
    role = f2[1]
    if len(f2) == 4 :
      company = f2[2] + ' ' + f2[3]
    else:
      company = f2[2]

    data = (name,role,company,number,mailid,web,address,state)
    if st.button('Extract Data'):
       datas = {'Name': name,
                'Role': role,
                'Company': company,
                'Number' : number,
                'Mail_id' : mailid,
                'Website' : web,
                'Address' : address,
                'State_and_Pincode' : state}
       df = pd.DataFrame(datas, index=[0])
       st.table(df)
    
    
    table_name = st.text_input('Give a table name')
    
    
    conn = sqlite3.connect('cards.db')
    if  st.button('Store into Database'):
          
          
          c = conn.cursor()
          c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name}
             (name varchar(50), role varchar(50), company varchar(50), number varchar(50),
              mailid varchar(50), web varchar(50), address varchar(50), state_and_pincode varchar(50))''')
          c.execute(f"INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?, ?, ?, ?)", data)
          conn.commit()
          st.write('Successfully Stored')
       
    if st.button('View Tables'):
          c = conn.cursor()
          c.execute("SELECT name FROM sqlite_master WHERE type='table'")
          table_names = [row[0] for row in c.fetchall()]
          radio = st.sidebar.write('List of Tables:' ,table_names)
          
       
    table_name = st.text_input('Inter Table Name')
    
    if st.button('View data'):
         
         df = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)
         st.dataframe(df)
         

Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.776s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.26s
your url is: https://spotty-birds-guess-34-145-248-83.loca.lt
